Question 1

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import nquad

In [ ]:

def gaussian_integral_integrand(*v, A, w):
    v = np.array(v)
    return np.exp(-0.5 * np.dot(v, np.dot(A, v)) + np.dot(v, w))

def verify_gaussian_integral_fast(A, w, L=10):
    N = len(w)
    try:
        A_inv = np.linalg.inv(A)
        v0 = A_inv.dot(w)
        sigma = np.sqrt(np.abs(np.diag(A_inv)))
    except:
        v0 = np.zeros(N)
        sigma = np.ones(N)
    bounds = [(v0[i] - L * sigma[i], v0[i] + L * sigma[i]) for i in range(N)]
    result, err = nquad(lambda *v: gaussian_integral_integrand(*v, A=A, w=w), bounds, opts={'limit': 50})
    detA = np.linalg.det(A)
    if detA > 0:
        closed_form = (2 * np.pi)**(N/2) / np.sqrt(detA) * np.exp(0.5 * np.dot(w, np.dot(A_inv, w)))
    else:
        closed_form = None
    return result, closed_form, err

A = np.array([[4, 2, 1],
              [2, 5, 3],
              [1, 3, 6]])
A_prime = np.array([[4, 2, 1],
                    [2, 1, 3],
                    [1, 3, 6]])
w = np.array([1, 2, 3])

result_A, closed_A, err_A = verify_gaussian_integral_fast(A, w)
print("For A:")
print("Numerical:", result_A)
print("Closed-form:", closed_A)
print("Error:", err_A)

result_Ap, closed_Ap, err_Ap = verify_gaussian_integral_fast(A_prime, w)
print("\nFor A':")
print("Numerical:", result_Ap)
print("Closed-form:", closed_Ap)
print("Error:", err_Ap)


For A:
Numerical: 4.275823658975703
Closed-form: 4.2758236590115155
Error: 5.031171865613e-08

For A':
Numerical: 0.0
Closed-form: None
Error: 3.7067656932499016e-08


In [ ]:
# 1.c

def vectorized_moments(A, w, L=10, M=51):
    N = len(w)
    A_inv = np.linalg.inv(A)
    v0 = A_inv.dot(w)
    sigma = np.sqrt(np.abs(np.diag(A_inv)))
    grids = [np.linspace(v0[i] - L * sigma[i], v0[i] + L * sigma[i], M) for i in range(N)]
    dvol = np.prod([grid[1] - grid[0] for grid in grids])
    mesh = np.meshgrid(*grids, indexing='ij')
    X = mesh[0].ravel()
    Y = mesh[1].ravel()
    Z = mesh[2].ravel()
    V = np.vstack([X, Y, Z]).T
    quad = np.sum(V * (V.dot(A)), axis=1)
    lin = V.dot(w)
    f = np.exp(-0.5 * quad + lin)
    Z_num = np.sum(f) * dvol
    moments = {}
    moments['v1']       = np.sum(X * f) * dvol / Z_num
    moments['v2']       = np.sum(Y * f) * dvol / Z_num
    moments['v3']       = np.sum(Z * f) * dvol / Z_num
    moments['v1v2']     = np.sum(X * Y * f) * dvol / Z_num
    moments['v2v3']     = np.sum(Y * Z * f) * dvol / Z_num
    moments['v1v3']     = np.sum(X * Z * f) * dvol / Z_num
    moments['v1^2v2']   = np.sum((X**2) * Y * f) * dvol / Z_num
    moments['v2v3^2']   = np.sum(Y * (Z**2) * f) * dvol / Z_num
    moments['v1^2v2^2'] = np.sum((X**2) * (Y**2) * f) * dvol / Z_num
    moments['v2^2v3^2'] = np.sum((Y**2) * (Z**2) * f) * dvol / Z_num
    return moments

def closed_form_moments(A, w):
    A_inv = np.linalg.inv(A)
    m = A_inv.dot(w)
    C = A_inv
    cf = {}
    cf['v1']       = m[0]
    cf['v2']       = m[1]
    cf['v3']       = m[2]
    cf['v1v2']     = m[0]*m[1] + C[0,1]
    cf['v2v3']     = m[1]*m[2] + C[1,2]
    cf['v1v3']     = m[0]*m[2] + C[0,2]
    cf['v1^2v2']   = m[0]**2 * m[1] + C[0,0]*m[1] + 2*m[0]*C[0,1]
    cf['v2v3^2']   = m[1] * m[2]**2 + C[2,2]*m[1] + 2*m[2]*C[1,2]
    cf['v1^2v2^2'] = (m[0]**2 + C[0,0])*(m[1]**2 + C[1,1]) + 2*(m[0]*m[1] + C[0,1])**2
    cf['v2^2v3^2'] = (m[1]**2 + C[1,1])*(m[2]**2 + C[2,2]) + 2*(m[1]*m[2] + C[1,2])**2
    return cf

A = np.array([[4, 2, 1],
              [2, 5, 3],
              [1, 3, 6]])
w = np.array([1, 2, 3])

num_moments = vectorized_moments(A, w, L=10, M=51)
cf_moments = closed_form_moments(A, w)

print("Numerical Moments:")
for k, v in num_moments.items():
    print(f"{k} = {v}")
print("\nClosed-Form Moments:")
for k, v in cf_moments.items():
    print(f"{k} = {v}")


Numerical Moments:
v1 = 0.08955223880597006
v2 = 0.1044776119402986
v3 = 0.4328358208955223
v1v2 = -0.12497215415460018
v2v3 = -0.1040320784139006
v1v3 = 0.0536867899309423
v1^2v2 = 0.009525772784551316
v2v3^2 = -0.08468129390915771
v1^2v2^2 = 0.1449191834042384
v2^2v3^2 = 0.16849831828214426

Closed-Form Moments:
v1 = 0.08955223880597016
v2 = 0.10447761194029859
v3 = 0.4328358208955224
v1v2 = -0.12497215415460013
v2v3 = -0.1040320784139006
v1v3 = 0.0536867899309423
v1^2v2 = 0.009525772784551318
v2v3^2 = -0.08468129390915768
v1^2v2^2 = 0.14509426051285187
v2^2v3^2 = 0.17258831406947536


Question 2

In [ ]:
#2.a
import os
import glob
import io
import pandas as pd

input_dir = "Local_density_of_states_near_band_edge"
output_dir = os.path.join(input_dir, "local density of states heatmap")
os.makedirs(output_dir, exist_ok=True)

file_list = sorted(glob.glob(os.path.join(input_dir, "*.txt")))
for file in file_list:
    # Read the file as text, remove commas, then load into pandas.
    with open(file, 'r') as f:
        content = f.read().replace(',', '')
    df = pd.read_csv(io.StringIO(content), delim_whitespace=True, header=None, dtype=float)
    data = df.values
    plt.figure(figsize=(8, 6))
    plt.imshow(data, cmap='viridis', origin='lower')
    plt.colorbar(label='LDOS Intensity')
    file_index = os.path.splitext(os.path.basename(file))[0]
    plt.title(f"Heatmap {file_index}")
    output_path = os.path.join(output_dir, f"{file_index}.png")
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()


In [ ]:
# 2.b
input_dir = "Local_density_of_states_near_band_edge"
output_dir = os.path.join(input_dir, "local density of states height")
os.makedirs(output_dir, exist_ok=True)

file_list = sorted(glob.glob(os.path.join(input_dir, "*.txt")))
for file in file_list:
    with open(file, 'r') as f:
        content = f.read().replace(',', '')
    df = pd.read_csv(io.StringIO(content), delim_whitespace=True, header=None, dtype=float)
    data = df.values

    nrows, ncols = data.shape
    X, Y = np.meshgrid(np.arange(ncols), np.arange(nrows))

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, data, cmap='viridis', edgecolor='none')
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5, label='LDOS Intensity')
    
    file_index = os.path.splitext(os.path.basename(file))[0]
    ax.set_title(f"Height Profile {file_index}")
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('LDOS')
    
    output_path = os.path.join(output_dir, f"{file_index}.png")
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()


In [21]:
# 2.c
# Average local density

input_dir = "Local_density_of_states_near_band_edge"
output_dir = os.path.join(input_dir, "subregion_ldos_analysis")
os.makedirs(output_dir, exist_ok=True)

r1, r2, c1, c2 = 50, 150, 50, 150

file_list = sorted(glob.glob(os.path.join(input_dir, "*.txt")))
file_indices = []
average_ldos = []

for file in file_list:
    with open(file, 'r') as f:
        content = f.read().replace(',', '')
    df = pd.read_csv(io.StringIO(content), delim_whitespace=True, header=None, dtype=float)
    data = df.values
    nrows, ncols = data.shape
    r1_actual = min(r1, nrows)
    r2_actual = min(r2, nrows)
    c1_actual = min(c1, ncols)
    c2_actual = min(c2, ncols)
    subregion = data[r1_actual:r2_actual, c1_actual:c2_actual]
    avg_val = np.mean(subregion)
    file_index = os.path.splitext(os.path.basename(file))[0]
    file_indices.append(file_index)
    average_ldos.append(avg_val)

plt.figure(figsize=(10, 6))
plt.plot(file_indices, average_ldos, marker='o', linestyle='-')
plt.xlabel('File Index (energy level)')
plt.ylabel('Average LDOS in Sub-region')
plt.title('Variation of Average LDOS in Selected Sub-region')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plot_path = os.path.join(output_dir, 'average_ldos_subregion.png')
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
plt.close()

print("Plot saved @:", plot_path)


Plot saved @: Local_density_of_states_near_band_edge/subregion_ldos_analysis/average_ldos_subregion.png
